In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import math

# Load the map image
map_image = cv2.imread('map.png', 0)
height, width = map_image.shape

# Define start and goal nodes
start_node = (200, 200)
goal_node = (600, 400)

# Define step size and number of iterations
step_size = 20
num_iterations = 5000

# Initialize the tree with the start node
tree = [start_node]

# Define functions
def generate_random_node():
    return (random.randint(0, width), random.randint(0, height))

def find_nearest_node(tree, node):
    distances = [(n[0]-node[0])**2 + (n[1]-node[1])**2 for n in tree]
    nearest_node_idx = np.argmin(distances)
    return tree[nearest_node_idx]

def generate_new_node(nearest_node, random_node, step_size):
    distance = np.sqrt((nearest_node[0]-random_node[0])**2 + (nearest_node[1]-random_node[1])**2)
    if distance <= step_size:
        return random_node
    else:
        theta = np.arctan2(random_node[1]-nearest_node[1], random_node[0]-nearest_node[0])
        return (nearest_node[0]+step_size*np.cos(theta), nearest_node[1]+step_size*np.sin(theta))

def is_collision(tree, new_node, map_image):
    for node in tree:
        # Draw a line between the new node and the existing node
        x1, y1 = int(node[0]), int(node[1])
        x2, y2 = int(new_node[0]), int(new_node[1])
        cv2.line(map_image, (x1, y1), (x2, y2), (255, 255, 255), 1)
        # Check if the line intersects with any obstacle
        if np.any(map_image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] == 0):
            return True
    return False

# Define a function to get the final path
def get_final_path(tree, goal_node):
    # Find the node in the tree closest to the goal node
    goal_node_idx = find_nearest_node(tree, goal_node)
    # Trace back from the goal node to the start node
    path = [goal_node]
    while goal_node_idx != 0:
        parent_node_idx = int((goal_node_idx-1)/2)
        path.insert(0, tree[parent_node_idx])
        goal_node_idx = parent_node_idx
    path.insert(0, start_node)
    return path

# Run RRT algorithm
for i in range(num_iterations):
    # Generate a random node
    random_node = generate_random_node()

    # Find the nearest node in the tree
    nearest_node = find_nearest_node(tree, random_node)

    # Generate a new node along the path to the random node
    new_node = generate_new_node(nearest_node, random_node, step_size)

    # Check if the new node collides with any obstacles
    if not is_collision(tree, new_node, map_image):
        # Add the new node to the tree
        tree.append(new_node)
        # Draw a line between the nearest node and the new node
        x1, y1 = int(nearest_node[0]), int(nearest_node[1])
        x2, y2 = int(new_node[0]), int(new_node[1])
        cv2.line(map_image, (x1, y1), (x2, y2), (255, 255, 255), 1)

        # Check if the new node is close to the goal node
        distance_to_goal = np.sqrt((new_node[0]-goal_node[0])**2 + (new_node[1]-goal_node[1])**2)
        if distance_to_goal < step_size:
            # Add the goal node to the tree
            tree.append(goal_node)
            # Draw a line between the new node and the goal node
            x1, y1 = int(new_node[0]), int(new_node[1])
            x2, y2 = int(goal_node[0]), int(goal_node[1])
            cv2.line(map_image, (x1, y1), (x2, y2), (255, 255, 255), 1)
            # Get the final path
            final_path = get_final_path(tree, goal_node)
            # Draw the final path
            for i in range(len(final_path)-1):
                x1, y1 = int(final_path[i][0]), int(final_path[i][1])
                x2, y2 = int(final_path[i+1][0]), int(final_path[i+1][1])
                cv2.line(map_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            break